In [1]:
import torch
import my_code.mysionna as sn
import tensorflow as tf
from my_code.mysionna.utils import GLOBAL_SEED_NUMBER
import numpy as np

def matrix_sqrt(tensor):
    r""" Computes the square root of a matrix.

    Given a batch of Hermitian positive semi-definite matrices
    :math:`\mathbf{A}`, returns matrices :math:`\mathbf{B}`,
    such that :math:`\mathbf{B}\mathbf{B}^H = \mathbf{A}`.

    The two inner dimensions are assumed to correspond to the matrix rows
    and columns, respectively.

    Args:
        tensor ([..., M, M]) : A tensor of rank greater than or equal
            to two.

    Returns:
        A tensor of the same shape and type as ``tensor`` containing
        the matrix square root of its last two dimensions.
    """
    if sn.config.xla_compat and not tensor.is_grad_enabled():
        s, u = torch.linalg.eigh(tensor)

        # Compute sqrt of eigenvalues
        s = torch.abs(s)
        s = torch.sqrt(s)
        s = s.type(dtype=u.dtype)

        # Matrix multiplication
        s = s.unsqueeze(-2)
        return torch.matmul(u * s, torch.conj(torch.transpose(u, -2, -1)))
    else:
        s, u = torch.linalg.eigh(tensor)

        # Compute sqrt of eigenvalues
        s = torch.abs(s)
        s = torch.sqrt(s)
        s = s.type(dtype=u.dtype)

        # Matrix multiplication
        s = s.unsqueeze(-2)
        return torch.matmul(u * s, torch.conj(torch.transpose(u, -2, -1)))
# Example usage:
tf.random.set_seed(GLOBAL_SEED_NUMBER)
tensor_tf = tf.random.uniform(shape=[3,3])
tensor_np = tensor_tf.numpy()
tensor = torch.from_numpy(tensor_np)
# tensor = torch.randn(3, 3, dtype=torch.float64)
tensor = tensor @ tensor.T  # Make it positive semi-definite

sqrt_tensor = matrix_sqrt(tensor)
print("Original tensor:\n", tensor)
print("Square root of the tensor:\n", sqrt_tensor)
print("Product of sqrt and its transpose:\n", sqrt_tensor @ sqrt_tensor.T)

2024-07-19 10:39:15.986268: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 10:39:15.986317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 10:39:15.988118: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-19 10:39:15.996705: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-19 10:39:17.255052: W tensorflow/compiler/tf2

Original tensor:
 tensor([[0.4503, 0.4728, 0.7179],
        [0.4728, 0.8853, 1.0855],
        [0.7179, 1.0855, 1.5643]])
Square root of the tensor:
 tensor([[0.5082, 0.2051, 0.3873],
        [0.2051, 0.7193, 0.5708],
        [0.3873, 0.5708, 1.0433]])
Product of sqrt and its transpose:
 tensor([[0.4503, 0.4728, 0.7179],
        [0.4728, 0.8853, 1.0855],
        [0.7179, 1.0855, 1.5643]])


2024-07-19 10:39:18.694683: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18425 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:09:00.0, compute capability: 8.6


In [2]:
# 测试例子
def test_matrix_sqrt_torch():
    # 创建 Hermitian 正半定矩阵
    A = torch.tensor([[4, 1+1j], [1-1j, 3]], dtype=torch.complex64)

    # 计算矩阵的平方根
    B = matrix_sqrt(A)

    # 验证 B * B^H 是否等于 A
    reconstructed_A = torch.matmul(B, B.transpose(-2, -1).conj())
    print("Original Matrix A:\n", A)
    print("Matrix Square Root B:\n", B)
    print("Reconstructed Matrix A:\n", reconstructed_A)

test_matrix_sqrt_torch()

Original Matrix A:
 tensor([[4.+0.j, 1.+1.j],
        [1.-1.j, 3.+0.j]])
Matrix Square Root B:
 tensor([[1.9621+0.0000j, 0.2740+0.2740j],
        [0.2740-0.2740j, 1.6882+0.0000j]])
Reconstructed Matrix A:
 tensor([[4.0000+8.2587e-10j, 1.0000+1.0000e+00j],
        [1.0000-1.0000e+00j, 3.0000+6.0175e-10j]])


In [3]:
import tensorflow as tf
import sionna as sn
def matrix_sqrt(tensor):
    r""" Computes the square root of a matrix.

    Given a batch of Hermitian positive semi-definite matrices
    :math:`\mathbf{A}`, returns matrices :math:`\mathbf{B}`,
    such that :math:`\mathbf{B}\mathbf{B}^H = \mathbf{A}`.

    The two inner dimensions are assumed to correspond to the matrix rows
    and columns, respectively.

    Args:
        tensor ([..., M, M]) : A tensor of rank greater than or equal
            to two.

    Returns:
        A tensor of the same shape and type as ``tensor`` containing
        the matrix square root of its last two dimensions.

    Note:
        If you want to use this function in Graph mode with XLA, i.e., within
        a function that is decorated with ``@tf.function(jit_compile=True)``,
        you must set ``sionna.config.xla_compat=true``.
        See :py:attr:`~sionna.config.xla_compat`.
    """
    if sn.config.xla_compat and not tf.executing_eagerly():
        s, u = tf.linalg.eigh(tensor)

        # Compute sqrt of eigenvalues
        s = tf.abs(s)
        s = tf.sqrt(s)
        s = tf.cast(s, u.dtype)

        # Matrix multiplication
        s = tf.expand_dims(s, -2)
        return tf.matmul(u*s, u, adjoint_b=True)
    else:
        return tf.linalg.sqrtm(tensor)

# 测试例子
def test_matrix_sqrt_tf():
    # 创建 Hermitian 正半定矩阵
    A = tf.constant([[4 + 0j, 1 + 1j], [1 - 1j, 3 + 0j]], dtype=tf.complex64)

    # 计算矩阵的平方根
    B = matrix_sqrt(A)

    # 验证 B * B^H 是否等于 A
    reconstructed_A = tf.matmul(B, tf.linalg.adjoint(B))
    
    # 打印结果
    print("Original Matrix A:\n", A.numpy())
    print("Matrix Square Root B:\n", B.numpy())
    print("Reconstructed Matrix A:\n", reconstructed_A.numpy())

test_matrix_sqrt_tf()

(4, 2, 2, 2, 2, 1, 10)
(4, 2, 2, 1)
Original Matrix A:
 [[4.+0.j 1.+1.j]
 [1.-1.j 3.+0.j]]
Matrix Square Root B:
 [[1.9621162 -2.4636604e-09j 0.27395147+2.7395147e-01j]
 [0.27395147-2.7395147e-01j 1.688165  +0.0000000e+00j]]
Reconstructed Matrix A:
 [[3.999999  -6.0174798e-10j 0.99999994+9.9999994e-01j]
 [0.99999994-9.9999994e-01j 2.9999998 -6.0174798e-10j]]


In [4]:
# 测试例子
def test_matrix_sqrt_tf():
    # 创建随机矩阵并使其为正半定矩阵
    tf.random.set_seed(GLOBAL_SEED_NUMBER)
    tensor = tf.random.uniform(shape=[3,3])
    tensor = tf.matmul(tensor, tensor, transpose_b=True)  # 使其为正半定矩阵

    # 计算矩阵的平方根
    sqrt_tensor = matrix_sqrt(tensor)

    # 验证 sqrt_tensor * sqrt_tensor^T 是否等于原始矩阵
    reconstructed_tensor = tf.matmul(sqrt_tensor, sqrt_tensor, transpose_b=True)
    
    # 打印结果
    tf.print("Original tensor:\n", tensor)
    tf.print("Square root of the tensor:\n", sqrt_tensor)
    tf.print("Product of sqrt and its transpose:\n", reconstructed_tensor)

test_matrix_sqrt_tf()

Original tensor:
 [[0.450283915 0.472771168 0.717919]
 [0.472771168 0.885265 1.08552921]
 [0.717919 1.08552921 1.56428695]]
Square root of the tensor:
 [[0.508175135 0.205055624 0.387291431]
 [0.205055609 0.719296813 0.570814967]
 [0.387291431 0.570815086 1.04329455]]
Product of sqrt and its transpose:
 [[0.450284421 0.472771764 0.717919767]
 [0.472771764 0.88526547 1.0855298]
 [0.717919767 1.0855298 1.56428814]]


In [5]:
import torch
h= torch.rand(size=(3,4,6,8,9))
print(h.dim())


5


In [6]:
import tensorflow as tf
h = tf.random.uniform(shape=[3,4,6,8,9])
print(tf.rank(h))

tf.Tensor(5, shape=(), dtype=int32)


## torch

In [7]:
"""Various classes for spatially correlated flat-fading channels."""

from abc import ABC, abstractmethod
import torch
import tensorflow as tf
import numpy as np
import torch.nn as nn

from my_code.mysionna.utils import expand_to_rank,matrix_sqrt

class SpatialCorrelation(ABC):
   # pylint: disable=line-too-long
    r"""Abstract class that defines an interface for spatial correlation functions.

    The :class:`~sionna.channel.FlatFadingChannel` model can be configured with a
    spatial correlation model.

    Input
    -----
    h : tf.complex
        Tensor of arbitrary shape containing spatially uncorrelated
        channel coefficients

    Output
    ------
    h_corr : tf.complex
        Tensor of the same shape and dtype as ``h`` containing the spatially
        correlated channel coefficients.
    """
    @abstractmethod
    def __call__(self, h, *args, **kwargs):
        return NotImplemented

class KroneckerModel(SpatialCorrelation):
    """Kronecker model for spatial correlation in PyTorch.

    Parameters
    ----------
    r_tx : [..., K, K], torch.complex
        Tensor containing the transmit correlation matrices.

    r_rx : [..., M, M], torch.complex
        Tensor containing the receive correlation matrices.

    Input
    -----
    h : [..., M, K], torch.complex
        Tensor containing spatially uncorrelated
        channel coefficients.

    Output
    ------
    h_corr : [..., M, K], torch.complex
        Tensor containing the spatially
        correlated channel coefficients.
    """
    def __init__(self, r_tx=None, r_rx=None):
        super().__init__()
        self.r_tx = r_tx
        self.r_rx = r_rx

    @property
    def r_tx(self):
        r"""Tensor containing the transmit correlation matrices.

        Note
        ----
        If you want to set this property in Graph mode with XLA, i.e., within
        a function that is decorated with ``@tf.function(jit_compile=True)``,
        you must set ``sionna.Config.xla_compat=true``.
        See :py:attr:`~sionna.Config.xla_compat`.
        """
        return self._r_tx
    
    @r_tx.setter
    def r_tx(self, value):
        self._r_tx = value
        if self._r_tx is not None:
            self._r_tx_sqrt = matrix_sqrt(value)
        else:
            self._r_tx_sqrt = None

    @property
    def r_rx(self):
        r"""Tensor containing the receive correlation matrices.

        Note
        ----
        If you want to set this property in Graph mode with XLA, i.e., within
        a function that is decorated with ``@tf.function(jit_compile=True)``,
        you must set ``sionna.Config.xla_compat=true``.
        See :py:attr:`~sionna.Config.xla_compat`.
        """
        return self._r_rx
    
    @r_rx.setter
    def r_rx(self, value):
        self._r_rx = value
        if self._r_rx is not None:
            self._r_rx_sqrt = matrix_sqrt(value)
        else:
            self._r_rx_sqrt = None
    
    def __call__(self, h, *args, **kwargs):
        if self._r_tx_sqrt is not None:
            r_tx_sqrt = expand_to_rank(self._r_tx_sqrt,h.dim(),0)
            h = torch.matmul(h, r_tx_sqrt.conj().transpose(-2, -1))

        
        if self._r_rx_sqrt is not None:
            r_rx_sqrt = expand_to_rank(self._r_rx_sqrt, h.dim(),0)
            h = torch.matmul(r_rx_sqrt, h)
        
        return h
# 测试例子
def test_kronecker_model():
    tf.random.set_seed(GLOBAL_SEED_NUMBER)

    # A = torch.tensor([[4, 1+1j], [1-1j, 3]], dtype=torch.complex64)
    # B = torch.tensor([[2, 0], [0, 2]], dtype=torch.complex64)
    
    # 创建 Hermitian 正半定矩阵
    A = torch.tensor([[4, 1+1j], [1-1j, 3]], dtype=torch.complex64)
    B = torch.tensor([[2, 0], [0, 2]], dtype=torch.complex64)    

    # 创建 KroneckerModel 实例
    model = KroneckerModel(r_tx=A, r_rx=B)

    # 创建未相关的通道系数矩阵
    h_tf_real = tf.random.normal(shape=[2,2],dtype=tf.float32)
    h_tf_img = tf.random.normal(shape=[2,2],dtype=tf.float32)
    h_np_real = h_tf_real.numpy()
    h_np_img = h_tf_img.numpy()

    h_torch_real = torch.tensor(h_np_real, dtype=torch.float32)
    h_torch_img = torch.tensor(h_np_img, dtype=torch.float32)
    h = torch.complex(h_torch_real,h_torch_img)
    
    # h = torch.randn(2, 2, dtype=torch.complex64)

    # 生成相关的通道系数矩阵
    h_corr = model(h)

    print("Uncorrelated channel coefficients:\n", h)
    print("Correlated channel coefficients:\n", h_corr)

test_kronecker_model()

class PerColumnModel(nn.Module):
    r"""Per-column model for spatial correlation.

    Given a batch of matrices :math:`\mathbf{H}\in\mathbb{C}^{M\times K}`
    and correlation matrices :math:`\mathbf{R}_k\in\mathbb{C}^{M\times M}`, k=1,\dots,K,
    this function will generate the output :math:`\mathbf{H}_\text{corr}\in\mathbb{C}^{M\times K}`,
    with columns

    .. math::

        \mathbf{h}^\text{corr}_k = \mathbf{R}^{\frac12}_k \mathbf{h}_k,\quad k=1, \dots, K

    where :math:`\mathbf{h}_k` is the kth column of :math:`\mathbf{H}`.
    Note that all :math:`\mathbf{R}_k\in\mathbb{C}^{M\times M}` must
    be positive semi-definite.

    This model is typically used to simulate a MIMO channel between multiple
    single-antenna users and a base station with multiple antennas.
    The resulting SIMO channel for each user has a different spatial correlation.

    Parameters
    ----------
    r_rx : [..., M, M], torch.complex
        Tensor containing the receive correlation matrices. If
        the rank of `r_rx` is smaller than that of the input `h`,
        it will be broadcast. For a typically use of this model, `r_rx`
        has shape [..., K, M, M], i.e., a different correlation matrix for each
        column of `h`.

    Input
    -----
    h : [..., M, K], torch.complex
        Tensor containing spatially uncorrelated
        channel coefficients.

    Output
    ------
    h_corr : [..., M, K], torch.complex
        Tensor containing the spatially
        correlated channel coefficients.
    """
    def __init__(self, r_rx):
        super().__init__()
        self.r_rx = r_rx

    @property
    def r_rx(self):
        """Tensor containing the receive correlation matrices.

        Note
        ----
        If you want to set this property in Graph mode with XLA, i.e., within
        a function that is decorated with ``@tf.function(jit_compile=True)``,
        you must set ``sionna.Config.xla_compat=true``.
        See :py:attr:`~sionna.Config.xla_compat`.
        """
        return self._r_rx
    
    @r_rx.setter
    def r_rx(self, value):
        self._r_rx = value
        if self._r_rx is not None:
            self._r_rx_sqrt = matrix_sqrt(value)
    
    def __call__(self, h):
        if self._r_rx is not None:
            h = h.transpose(-2, -1)
            h = h.unsqueeze(-1)
            r_rx_sqrt = expand_to_rank(self._r_rx_sqrt, h.dim(), 0)
            h = torch.matmul(r_rx_sqrt, h)
            h = h.squeeze(-1)
            h = h.transpose(-2, -1)

        return h      

def test_PerColumnModel():
    tf.random.set_seed(GLOBAL_SEED_NUMBER)
    # Define input parameters
    M, K = 4, 3  # Dimensions of the matrices
    h_tf = tf.complex(tf.random.normal([M, K]), tf.random.normal([M, K]))  # Random complex matrix
    r_rx_tf = tf.complex(tf.eye(M, batch_shape=[K]), tf.zeros([K, M, M]))  # Identity matrices as correlation matrices

    r_rx_np = r_rx_tf.numpy()
    r_rx = torch.from_numpy(r_rx_np)

    h_np = h_tf.numpy()
    h = torch.from_numpy(h_np)
    # Initialize the PerColumnModel
    model = PerColumnModel(r_rx)

    # Get the correlated channel coefficients
    h_corr = model(h)

    # Print the input and output matrices
    print("Input h:\n", h)
    print("Output h_corr:\n", h_corr)

test_PerColumnModel()
    


Uncorrelated channel coefficients:
 tensor([[ 0.1605+0.6497j, -1.6598+0.3279j],
        [-1.2321-0.7520j,  0.5972-0.2143j]])
Correlated channel coefficients:
 tensor([[-0.0706+2.5730j, -4.1521+1.0968j],
        [-3.2707-2.4010j,  1.2397-1.2803j]])
Input h:
 tensor([[ 0.1605+0.6497j, -1.6598+0.3279j, -1.2321-0.7520j],
        [ 0.5972-0.2143j,  1.0610+0.5260j, -1.3278+1.1993j],
        [-0.2791-1.2921j, -0.0214+0.1150j, -1.5022-0.4270j],
        [ 0.3066+0.6561j,  0.5355+0.0611j, -1.3167+0.7816j]])
Output h_corr:
 tensor([[ 0.1605+0.6497j, -1.6598+0.3279j, -1.2321-0.7520j],
        [ 0.5972-0.2143j,  1.0610+0.5260j, -1.3278+1.1993j],
        [-0.2791-1.2921j, -0.0214+0.1150j, -1.5022-0.4270j],
        [ 0.3066+0.6561j,  0.5355+0.0611j, -1.3167+0.7816j]])


## tensorflow

In [8]:
from abc import ABC, abstractmethod
import tensorflow as tf
from tensorflow.experimental.numpy import swapaxes
from sionna.utils import expand_to_rank, matrix_sqrt

class SpatialCorrelation(ABC):
    # pylint: disable=line-too-long
    r"""Abstract class that defines an interface for spatial correlation functions.

    The :class:`~sionna.channel.FlatFadingChannel` model can be configured with a
    spatial correlation model.

    Input
    -----
    h : tf.complex
        Tensor of arbitrary shape containing spatially uncorrelated
        channel coefficients

    Output
    ------
    h_corr : tf.complex
        Tensor of the same shape and dtype as ``h`` containing the spatially
        correlated channel coefficients.
    """
    @abstractmethod
    def __call__(self, h, *args, **kwargs):
        return NotImplemented

class KroneckerModel(SpatialCorrelation):
    # pylint: disable=line-too-long
    r"""Kronecker model for spatial correlation.

    Given a batch of matrices :math:`\mathbf{H}\in\mathbb{C}^{M\times K}`,
    :math:`\mathbf{R}_\text{tx}\in\mathbb{C}^{K\times K}`, and
    :math:`\mathbf{R}_\text{rx}\in\mathbb{C}^{M\times M}`, this function
    will generate the following output:

    .. math::

        \mathbf{H}_\text{corr} = \mathbf{R}^{\frac12}_\text{rx} \mathbf{H} \mathbf{R}^{\frac12}_\text{tx}

    Note that :math:`\mathbf{R}_\text{tx}\in\mathbb{C}^{K\times K}` and :math:`\mathbf{R}_\text{rx}\in\mathbb{C}^{M\times M}`
    must be positive semi-definite, such as the ones generated by
    :meth:`~sionna.channel.exp_corr_mat`.

    Parameters
    ----------
    r_tx : [..., K, K], tf.complex
        Tensor containing the transmit correlation matrices. If
        the rank of ``r_tx`` is smaller than that of the input ``h``,
        it will be broadcast.

    r_rx : [..., M, M], tf.complex
        Tensor containing the receive correlation matrices. If
        the rank of ``r_rx`` is smaller than that of the input ``h``,
        it will be broadcast.

    Input
    -----
    h : [..., M, K], tf.complex
        Tensor containing spatially uncorrelated
        channel coeffficients.

    Output
    ------
    h_corr : [..., M, K], tf.complex
        Tensor containing the spatially
        correlated channel coefficients.
    """
    def __init__(self, r_tx=None, r_rx=None):
        super().__init__()
        self.r_tx = r_tx
        self.r_rx = r_rx

    @property
    def r_tx(self):
        r"""Tensor containing the transmit correlation matrices.

        Note
        ----
        If you want to set this property in Graph mode with XLA, i.e., within
        a function that is decorated with ``@tf.function(jit_compile=True)``,
        you must set ``sionna.Config.xla_compat=true``.
        See :py:attr:`~sionna.Config.xla_compat`.
        """
        return self._r_tx

    @r_tx.setter
    def r_tx(self, value):
        self._r_tx = value
        if self._r_tx is not None:
            self._r_tx_sqrt = matrix_sqrt(value)
        else:
            self._r_tx_sqrt = None

    @property
    def r_rx(self):
        r"""Tensor containing the receive correlation matrices.

        Note
        ----
        If you want to set this property in Graph mode with XLA, i.e., within
        a function that is decorated with ``@tf.function(jit_compile=True)``,
        you must set ``sionna.Config.xla_compat=true``.
        See :py:attr:`~sionna.Config.xla_compat`.
        """
        return self._r_rx

    @r_rx.setter
    def r_rx(self, value):
        self._r_rx = value
        if self._r_rx is not None:
            self._r_rx_sqrt = matrix_sqrt(value)
        else:
            self._r_rx_sqrt = None

    def __call__(self, h):
        if self._r_tx_sqrt is not None:
            r_tx_sqrt = expand_to_rank(self._r_tx_sqrt, tf.rank(h), 0)
            h = tf.matmul(h, r_tx_sqrt, adjoint_b=True)

        if self._r_rx_sqrt is not None:
            r_rx_sqrt = expand_to_rank(self._r_rx_sqrt, tf.rank(h), 0)
            h = tf.matmul(r_rx_sqrt, h)

        return h
    
def test_kronecker_model():
    # 设置随机种子
    tf.random.set_seed(GLOBAL_SEED_NUMBER)

    # 创建 Hermitian 正半定矩阵
    A_real = tf.constant([[4, 1], [1, 3]], dtype=tf.float32)
    A_imag = tf.constant([[0, 1], [-1, 0]], dtype=tf.float32)
    A = tf.complex(A_real, A_imag)

    B_real = tf.constant([[2, 0], [0, 2]], dtype=tf.float32)
    B_imag = tf.constant([[0, 0], [0, 0]], dtype=tf.float32)
    B = tf.complex(B_real, B_imag)

    # 创建 KroneckerModel 实例
    model = KroneckerModel(r_tx=A, r_rx=B)

    # 创建未相关的通道系数矩阵
    h_real = tf.random.normal(shape=[2, 2], dtype=tf.float32)
    h_imag = tf.random.normal(shape=[2, 2], dtype=tf.float32)
    h = tf.complex(h_real, h_imag)

    # 生成相关的通道系数矩阵
    h_corr = model(h)

    print("Uncorrelated channel coefficients:\n", h)
    print("Correlated channel coefficients:\n", h_corr)

test_kronecker_model()
print("----------------------------------")
class PerColumnModel(SpatialCorrelation):
        # pylint: disable=line-too-long
    r"""Per-column model for spatial correlation.

    Given a batch of matrices :math:`\mathbf{H}\in\mathbb{C}^{M\times K}`
    and correlation matrices :math:`\mathbf{R}_k\in\mathbb{C}^{M\times M}, k=1,\dots,K`,
    this function will generate the output :math:`\mathbf{H}_\text{corr}\in\mathbb{C}^{M\times K}`,
    with columns

    .. math::

        \mathbf{h}^\text{corr}_k = \mathbf{R}^{\frac12}_k \mathbf{h}_k,\quad k=1, \dots, K

    where :math:`\mathbf{h}_k` is the kth column of :math:`\mathbf{H}`.
    Note that all :math:`\mathbf{R}_k\in\mathbb{C}^{M\times M}` must
    be positive semi-definite, such as the ones generated
    by :meth:`~sionna.channel.one_ring_corr_mat`.

    This model is typically used to simulate a MIMO channel between multiple
    single-antenna users and a base station with multiple antennas.
    The resulting SIMO channel for each user has a different spatial correlation.

    Parameters
    ----------
    r_rx : [..., M, M], tf.complex
        Tensor containing the receive correlation matrices. If
        the rank of ``r_rx`` is smaller than that of the input ``h``,
        it will be broadcast. For a typically use of this model, ``r_rx``
        has shape [..., K, M, M], i.e., a different correlation matrix for each
        column of ``h``.

    Input
    -----
    h : [..., M, K], tf.complex
        Tensor containing spatially uncorrelated
        channel coeffficients.

    Output
    ------
    h_corr : [..., M, K], tf.complex
        Tensor containing the spatially
        correlated channel coefficients.
    """
    def __init__(self, r_rx):
        super().__init__()
        self.r_rx = r_rx

    @property
    def r_rx(self):
        """Tensor containing the receive correlation matrices.

        Note
        ----
        If you want to set this property in Graph mode with XLA, i.e., within
        a function that is decorated with ``@tf.function(jit_compile=True)``,
        you must set ``sionna.Config.xla_compat=true``.
        See :py:attr:`~sionna.Config.xla_compat`.
        """

        return self._r_rx

    @r_rx.setter
    def r_rx(self, value):
        self._r_rx = value
        if self._r_rx is not None:
            self._r_rx_sqrt = matrix_sqrt(value)

    def __call__(self, h):
        if self._r_rx is not None:
            h = swapaxes(h, -2, -1)
            h = tf.expand_dims(h, -1)
            r_rx_sqrt = expand_to_rank(self._r_rx_sqrt, tf.rank(h), 0)
            h = tf.matmul(r_rx_sqrt, h)
            h = tf.squeeze(h, -1)
            h = swapaxes(h, -2, -1)

        return h
def test_PerColumnModel():
    tf.random.set_seed(GLOBAL_SEED_NUMBER)
    # Define input parameters
    M, K = 4, 3  # Dimensions of the matrices
    h = tf.complex(tf.random.normal([M, K]), tf.random.normal([M, K]))  # Random complex matrix
    r_rx = tf.complex(tf.eye(M, batch_shape=[K]), tf.zeros([K, M, M]))  # Identity matrices as correlation matrices

    # Initialize the PerColumnModel
    model = PerColumnModel(r_rx)

    # Get the correlated channel coefficients
    h_corr = model(h)

    # Print the input and output matrices
    print("Input h:\n", h)
    print("Output h_corr:\n", h_corr)

# Run the test
test_PerColumnModel()

Uncorrelated channel coefficients:
 tf.Tensor(
[[ 0.16052227+0.64973587j -1.6597689 +0.32791495j]
 [-1.2321332 -0.75198144j  0.5971658 -0.21430095j]], shape=(2, 2), dtype=complex64)
Correlated channel coefficients:
 tf.Tensor(
[[-0.07056929+2.5730007j -4.152109  +1.0967876j]
 [-3.2706544 -2.40102j    1.2396659 -1.2803249j]], shape=(2, 2), dtype=complex64)
----------------------------------
Input h:
 tf.Tensor(
[[ 0.16052227+0.64973587j -1.6597689 +0.32791495j -1.2321332 -0.75198144j]
 [ 0.5971658 -0.21430095j  1.0609884 +0.52599317j -1.3277572 +1.1992904j ]
 [-0.27911443-1.292074j   -0.02141875+0.1150163j  -1.502249  -0.42695856j]
 [ 0.3066489 +0.6561277j   0.5355358 +0.06107097j -1.3167298 +0.78159803j]], shape=(4, 3), dtype=complex64)
Output h_corr:
 tf.Tensor(
[[ 0.16052227+0.64973587j -1.6597689 +0.32791495j -1.2321332 -0.75198144j]
 [ 0.5971658 -0.21430095j  1.0609884 +0.52599317j -1.3277572 +1.1992904j ]
 [-0.27911443-1.292074j   -0.02141875+0.1150163j  -1.502249  -0.42695856j]
 

In [63]:
import tensorflow as tf
import warnings

from sionna import PI
from sionna.utils import expand_to_rank

def exp_corr_mat(a, n, dtype=tf.complex64):
    # Cast to desired output dtype and expand last dimension for broadcasting
    a = tf.cast(a, dtype=dtype)
    a = tf.expand_dims(a, -1)

    # Check that a is valid
    msg = "The absolute value of the elements of `a` must be smaller than one"
    tf.debugging.assert_less(tf.abs(a), tf.cast(1, a.dtype.real_dtype), msg)

    # Vector of exponents, adapt dtype and dimensions for broadcasting
    exp = tf.range(0, n)
    exp = tf.cast(exp, dtype=dtype)
    exp = expand_to_rank(exp, tf.rank(a), 0)

    # First column of R
    col = tf.math.pow(a, exp)

    # For a=0, one needs to remove the resulting nans due to 0**0=nan
    cond = tf.math.is_nan(tf.math.real(col))
    col = tf.where(cond, tf.ones_like(col), col)

    # First row of R (equal to complex-conjugate of the first column)
    row = tf.math.conj(col)

    # Create Toeplitz operator
    operator = tf.linalg.LinearOperatorToeplitz(col, row)

    # Generate dense tensor from operator
    r = operator.to_dense()

    return r
# 测试例子
a = tf.constant([0.5 + 0.5j,0.1+0.6j], dtype=tf.complex64)
n = 4
result = exp_corr_mat(a, n)

print("Result:")
print(result)

Result:
tf.Tensor(
[[[ 1.0000000e+00+0.j          4.9999997e-01-0.49999997j
   -2.1855694e-08-0.5j        -2.4999999e-01-0.24999999j]
  [ 4.9999997e-01+0.49999997j  1.0000000e+00+0.j
    4.9999997e-01-0.49999997j -2.1855694e-08-0.5j       ]
  [-2.1855694e-08+0.5j         4.9999997e-01+0.49999997j
    1.0000000e+00+0.j          4.9999997e-01-0.49999997j]
  [-2.4999999e-01+0.24999999j -2.1855694e-08+0.5j
    4.9999997e-01+0.49999997j  1.0000000e+00+0.j        ]]

 [[ 1.0000000e+00+0.j          1.0000001e-01-0.6j
   -3.4999999e-01-0.12j       -1.0700002e-01+0.19799998j]
  [ 1.0000001e-01+0.6j         1.0000000e+00+0.j
    1.0000001e-01-0.6j        -3.4999999e-01-0.12j      ]
  [-3.4999999e-01+0.12j        1.0000001e-01+0.6j
    1.0000000e+00+0.j          1.0000001e-01-0.6j       ]
  [-1.0700002e-01-0.19799998j -3.4999999e-01+0.12j
    1.0000001e-01+0.6j         1.0000000e+00+0.j        ]]], shape=(2, 4, 4), dtype=complex64)


In [64]:
import torch

from my_code.mysionna.utils import expand_to_rank
from my_code.mysionna import PI

def exp_corr_mat(a, n, dtype=torch.complex64):
    r"""Generate exponential correlation matrices.

    This function computes for every element :math:`a` of a complex-valued
    tensor :math:`\mathbf{a}` the corresponding :math:`n\times n` exponential
    correlation matrix :math:`\mathbf{R}(a,n)`, defined as (Eq. 1, [MAL2018]_):

    .. math::
        \mathbf{R}(a,n)_{i,j} = \begin{cases}
                    1 & \text{if } i=j\\
                    a^{i-j}  & \text{if } i>j\\
                    (a^\star)^{j-i}  & \text{if } j<i, j=1,\dots,n\\
                  \end{cases}

    where :math:`|a|<1` and :math:`\mathbf{R}\in\mathbb{C}^{n\times n}`.

    Input
    -----
    a : [n_0, ..., n_k], torch.complex
        A tensor of arbitrary rank whose elements
        have an absolute value smaller than one.

    n : int
        Number of dimensions of the output correlation matrices.

    dtype : torch.complex64, torch.complex128
        The dtype of the output.

    Output
    ------
    R : [n_0, ..., n_k, n, n], torch.complex
        A tensor of the same dtype as the input tensor :math:`\mathbf{a}`.
    """
    if dtype == torch.complex32:
        real_dtype = torch.float16
    elif dtype == torch.complex64:
        real_dtype = torch.float32
    elif dtype == torch.complex128:
        real_dtype = torch.float64
    else:
        raise TypeError("Not found comfortable type")

    # Cast to desired output dtype and expand last dimension for broadcasting
    a = a.to(dtype=dtype)
    a = a.unsqueeze(-1)

    # Check that a is valid
    if not torch.all(torch.abs(a) < 1):
        raise ValueError("The absolute value of the elements of `a` must be smaller than one")

    # Vector of exponents, adapt dtype and dimensions for broadcasting
    exp = torch.arange(0, n, dtype=real_dtype)
    exp = exp.to(dtype=dtype) 
    exp = expand_to_rank(exp, a.dim(),0)

    # First column of R
    col = torch.pow(a, exp)

    # For a=0, one needs to remove the resulting nans due to 0**0=nan
    col = torch.where(torch.isnan(col.real), torch.ones_like(col), col)

    # First row of R (equal to complex-conjugate of the first column)
    row = torch.conj(col)

    # Create Toeplitz matrix manually
    R = torch.zeros(*a.shape[:-1], n, n,dtype=dtype)
    for i in range(n):
        for j in range(n):
            if i == j:
                R[..., i, j] = 1
            elif i > j:
                R[..., i, j] = col[..., i-j]
            else:
                R[..., i, j] = row[..., j-i]

    return R
# 测试例子
a = torch.tensor([0.5 + 0.5j,0.1+0.6j], dtype=torch.complex128)
n = 4
result = exp_corr_mat(a, n)

print("Result:")
print(result)
print(result.shape)

Result:
tensor([[[ 1.0000e+00+0.0000j,  5.0000e-01-0.5000j, -2.1856e-08-0.5000j,
          -2.5000e-01-0.2500j],
         [ 5.0000e-01+0.5000j,  1.0000e+00+0.0000j,  5.0000e-01-0.5000j,
          -2.1856e-08-0.5000j],
         [-2.1856e-08+0.5000j,  5.0000e-01+0.5000j,  1.0000e+00+0.0000j,
           5.0000e-01-0.5000j],
         [-2.5000e-01+0.2500j, -2.1856e-08+0.5000j,  5.0000e-01+0.5000j,
           1.0000e+00+0.0000j]],

        [[ 1.0000e+00+0.0000j,  1.0000e-01-0.6000j, -3.5000e-01-0.1200j,
          -1.0700e-01+0.1980j],
         [ 1.0000e-01+0.6000j,  1.0000e+00+0.0000j,  1.0000e-01-0.6000j,
          -3.5000e-01-0.1200j],
         [-3.5000e-01+0.1200j,  1.0000e-01+0.6000j,  1.0000e+00+0.0000j,
           1.0000e-01-0.6000j],
         [-1.0700e-01-0.1980j, -3.5000e-01+0.1200j,  1.0000e-01+0.6000j,
           1.0000e+00+0.0000j]]])
torch.Size([2, 4, 4])
